In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import string
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score
from tqdm import tqdm


In [5]:
# 1. Load and Prepare Dataset

df_train = pd.read_csv(r"\TalkHealth\Datasets\train.csv\train.csv")

def clean_text(text):
    text = text.lower()
    leet_dict = {'0': 'o', '1': 'i', '3': 'e', '4': 'a', '5': 's', '7': 't'}
    for leet, letter in leet_dict.items():
        text = text.replace(leet, letter)
    text = re.sub(r'(\w)\.(\w)', r'\1\2', text)
    text = ''.join(c for c in text if c in string.printable)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Clean comments
df_train['comment_text'] = df_train['comment_text'].apply(clean_text)

# Prepare labels
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

#Subsample to 20k for quick training
df_train = df_train.sample(n=20000, random_state=42).reset_index(drop=True)

# Stratified split
stratify_labels = df_train[label_cols].sum(axis=1)
X_train, X_val, Y_train, Y_val = train_test_split(
    df_train['comment_text'].tolist(),
    df_train[label_cols].values,
    test_size=0.1,
    random_state=42,
    stratify=stratify_labels
)

# Create a small warmup subset (5k samples)
df_warmup = df_train.sample(n=5000, random_state=123).reset_index(drop=True)
X_warmup = df_warmup['comment_text'].tolist()
Y_warmup = df_warmup[label_cols].values


In [48]:
# Load model and tokenizer (distilroberta-base)

model_checkpoint = "distilroberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=6)

# Dataset preparation

class ToxicCommentsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=256,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(label, dtype=torch.float)
        return item

warmup_dataset = ToxicCommentsDataset(X_warmup, Y_warmup, tokenizer)
train_dataset = ToxicCommentsDataset(X_train, Y_train, tokenizer)
val_dataset = ToxicCommentsDataset(X_val, Y_val, tokenizer)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
# Metrics for evaluation

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    preds = (preds > 0.5).astype(int)

    micro_f1 = f1_score(labels, preds, average='micro', zero_division=0)
    macro_f1 = f1_score(labels, preds, average='macro', zero_division=0)
    micro_precision = precision_score(labels, preds, average='micro', zero_division=0)
    macro_precision = precision_score(labels, preds, average='macro', zero_division=0)
    micro_recall = recall_score(labels, preds, average='micro', zero_division=0)
    macro_recall = recall_score(labels, preds, average='macro', zero_division=0)

    return {
        'micro_f1': micro_f1,
        'macro_f1': macro_f1,
        'micro_precision': micro_precision,
        'macro_precision': macro_precision,
        'micro_recall': micro_recall,
        'macro_recall': macro_recall
    }


In [ ]:
# Warm-up training arguments

warmup_training_args = TrainingArguments(
    output_dir="./warmup_checkpoint",
    eval_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./warmup_logs',
    save_total_limit=1,
)

# Warm-up trainer initialization and training

warmup_trainer = Trainer(
    model=model,
    args=warmup_training_args,
    train_dataset=warmup_dataset,
)

model.to(device)

warmup_trainer.train()

model.save_pretrained("./warmup_distilroberta_model")
tokenizer.save_pretrained("./warmup_distilroberta_model")

Step,Training Loss


In [68]:
# Load warm-up model for full fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("./warmup_distilroberta_model")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
           

In [ ]:
# Full training arguments

training_args = TrainingArguments(
    output_dir="./warmup_distilroberta_model",
    eval_strategy="steps",
    eval_steps=100,
    save_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_dir='./final_logs',
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    save_total_limit=2,
    fp16=True
)
# Full fine-tuning trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

model.save_pretrained("./distilroberta_trained")
tokenizer.save_pretrained("./distilroberta_trained")

Step,Training Loss,Validation Loss,Micro F1,Macro F1,Micro Precision,Macro Precision,Micro Recall,Macro Recall
100,No log,0.043752,0.750623,0.391622,0.820163,0.406743,0.691954,0.377593
200,No log,0.044952,0.752451,0.393293,0.805774,0.404392,0.705747,0.383399


KeyboardInterrupt: 

In [101]:
#TESTING

# Load model and tokenizer
model_checkpoint = './distilroberta_trained'
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
model.eval()

# Load test data
df_test = pd.read_csv(r"\TalkHealth\Datasets\test.csv\test.csv")
df_labels = pd.read_csv(r"\TalkHealth\Datasets\test.csv\test.csv")


# Some test labels may be -1 (unlabeled), so filter
df = df_test.merge(df_labels, on='id')
df = df[df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].min(axis=1) >= 0]
df = df.reset_index(drop=True)


stratify_labels = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1)
df_sampled, _ = train_test_split(
    df,
    train_size=20000,
    random_state=42,
    stratify=stratify_labels
)
df = df_sampled.reset_index(drop=True)

texts = df['comment_text'].astype(str).apply(clean_text).tolist()
labels = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values


In [105]:
# Define test dataset
class TestDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=256):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        return item

    def __len__(self):
        return len(self.texts)

# Prepare dataset and dataloader
test_dataset = TestDataset(texts, tokenizer)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)


In [119]:
model.to(device)

all_logits = []

model.eval()
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        all_logits.append(logits.cpu())  # move back to CPU for later stacking

logits = torch.cat(all_logits)

Y_true = labels

Using device: cuda


Predicting:   0%|▏                                                                    | 4/1250 [00:02<15:14,  1.36it/s]


KeyboardInterrupt: 

In [117]:
# Convert to probabilities

probs = torch.sigmoid(logits).cpu().numpy()

# Find the best threshold for each lable

num_labels = probs.shape[1]
best_thresholds = []

for label_idx in range(num_labels):
    best_f1 = 0
    best_thresh = 0.5  # default starting value

    for threshold in np.arange(0.3, 0.8, 0.01):
        preds = (probs[:, label_idx] > threshold).astype(int)
        f1 = f1_score(Y_true[:, label_idx], preds, zero_division=0)

        if f1 > best_f1:
            best_f1 = f1
            best_thresh = threshold

    best_thresholds.append(best_thresh)
    print(f"Label {label_idx}: Best threshold = {best_thresh:.2f}, Best F1 = {best_f1:.4f}")

print("\nBest thresholds per label:", best_thresholds)

Label 0: Best threshold = 0.79, Best F1 = 0.6990
Label 1: Best threshold = 0.44, Best F1 = 0.4326
Label 2: Best threshold = 0.79, Best F1 = 0.7084
Label 3: Best threshold = 0.50, Best F1 = 0.0000
Label 4: Best threshold = 0.66, Best F1 = 0.6607
Label 5: Best threshold = 0.30, Best F1 = 0.1550

Best thresholds per label: [0.7900000000000005, 0.4400000000000001, 0.7900000000000005, 0.5, 0.6600000000000004, 0.3]


In [124]:
final_preds = np.zeros_like(probs)

for label_idx in range(probs.shape[1]):
    final_preds[:, label_idx] = (probs[:, label_idx] > best_thresholds[label_idx]).astype(int)


# Evaluate the results
micro_f1 = f1_score(Y_true, final_preds, average='micro', zero_division=0)
macro_f1 = f1_score(Y_true, final_preds, average='macro', zero_division=0)
micro_precision = precision_score(Y_true, final_preds, average='micro', zero_division=0)
macro_precision = precision_score(Y_true, final_preds, average='macro', zero_division=0)
micro_recall = recall_score(Y_true, final_preds, average='micro', zero_division=0)
macro_recall = recall_score(Y_true, final_preds, average='macro', zero_division=0)

print(f"Micro F1: {micro_f1:.4f}")
print(f"Macro F1: {macro_f1:.4f}")
print(f"Micro Precision: {micro_precision:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Micro Recall: {micro_recall:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")

Micro F1: 0.6635
Macro F1: 0.4426
Micro Precision: 0.6440
Macro Precision: 0.4905
Micro Recall: 0.6841
Macro Recall: 0.4623
